In [ ]:
# import general-use Python libraries
import random
import pandas as pd
import geopandas as gpd
import numpy as np
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score

# import custom functions from src
from hpc4ag.preprocessing import split_train_test, create_patches
from hpc4ag.modeling import (compile_model, generate_X_and_y,
                             assign_class_weight, apply_scaler_train,
                             apply_scaler_test)
from hpc4ag.visualization import (map_training_labels, plot_sample_distribution,
                                  map_timeseries_oneband, plot_indices_temporal,
                                  plot_confusion_matrix)
from hpc4ag.utils import download_file, get_tmpdest, load_pickle, save_pickle

### **Configurations**
We use the following data formats to store our data:
- `.shp` is one of the most common formats to store `spatial vector data` (geographic features that can be represented by points, lines, or polygons). For more information https://doc.arcgis.com/en/arcgis-online/reference/shapefiles.htm;
- `.sr6d` is a custom data format used here to serialize a Python object structure and store data as Python `pickles`. In our case we are storing Python dictionaries, which are structured as `{key : value}` pairs. For more information on pickles https://docs.python.org/3/library/pickle.html and dictionaries https://docs.python.org/3/tutorial/datastructures.html#dictionaries.

Data Repository: all data files are stored in s3 bucket configured for public "read" access (`https://s3.msi.umn.edu/hpc4ag/csb`). We will download the data on the fly and generate local temporary files and directories to store and access data during the process.

In [ ]:
shapefile_data_filename = "norman-mn-csb-2022-training-set-wsg84.zip"

raw_data_filename = "csb_v2m58_plf_y2022_s{}.sr6d"
field_ids = list(range(0,280))

train_filename = "train_patches.sr6d"
test_filename = "test_patches.sr6d"

crop_list = ["Corn", "Soybeans", "Sugarbeets", "Spring Wheat"]

### **Step 0. Formulate your research objective and understand your data**

**Research Objective:** Predict crop types (Corn, Soybeans, Sugarbeets, Spring Wheat) in one county in Minnesota using time series Sentinel data and a neural network model.  
**Problem Type:** Classification  
**Study Period:** 2022  
**Data:**
- observations - Sentinel data
- labels - crop types

Ideally, crop type labels should be ground truth data. However, this type of data is often confidential, and therefore for this tutorial we use a modeled data sample - USDA's `Crop Sequence Boundaries (CSB)` to "imitate" real crop type labels in 2022. For more information on this dataset follow https://www.nass.usda.gov/Research_and_Science/Crop-Sequence-Boundaries/index.php.

Note that we have only 280 agricultural field labels, but each field is comprised of many individual observations (pixels). Sentinel data files have a multi-dimensional dictionary structure to store spectral information needed for the spatiotemporal analysis in an organized way. The `grid` component of one data file is a 4D array with shape (time, band, row, column). Sentinel data have been sampled/interpolated at a 5-day interval.

In [ ]:
training_labels_gdf = gpd.read_file(download_file(bucket_path="csb/", filename=shapefile_data_filename))
print ("Count of agricultural fields:", len(training_labels_gdf))
training_labels_gdf.head()

Above we have printed the head of our `GeoDataFrame` to explore the data attributes (`GeoDataFrame` is a Python's data structure provided by the `geopandas` library to handle spatial vector data).
- `CSBID` - unique field ID;
- `CSBACRES` - field area in acres;
- `CNTY` - county name;
- `R22` - crop type in 2022, stored as a numeric value;
- `CSB 2022` - crop type in 2022, stored as a text value;
- `geometry` column enables spatial operations.

In [ ]:
map_training_labels(gdf=training_labels_gdf, column_name="CSB 2022")

In [ ]:
plot_sample_distribution(gdf=training_labels_gdf, column_name="CSB 2022")

In [ ]:
# get a random item from field ids list
random_filepath = download_file(bucket_path="csb/", filename=raw_data_filename.format(random.choice(field_ids)))
print("Random field :",  random_filepath)

- Below we load the data from a Python pickle (described at the beginning) into a Python dictionary and show how to unpack the dictionary structure to read important information about the data, such as the dates of observations.

In [ ]:
# load random data file and check metadata
field_data = load_pickle(filepath=random_filepath)
print ("Overview of data components:", list(field_data.keys()))
print ("Bands:", field_data["grid_info"])
grid = field_data["samples"][0]["grids"][0]
stack = grid["stack"]
dates = grid["dates"]
print ("Dates:", dates)

In [ ]:
# visualize grid sample
map_timeseries_oneband(array=stack, band_index=8, dates=dates)

### **Step 1. Split input data into train and test sets**

We split the data at a **field** level (NOT **pixel** level) to ensure that pixels from the same field do not appear in both train and test sets. We use the default  80% (train) to 20% (test) ratio.

In [ ]:
train_ids, test_ids = split_train_test(objects=field_ids, test_size=0.2)

# Print the count of train and test IDs
print("Count of train fields:", len(train_ids))
print("Count test fields:", len(test_ids))

### **Step 2. Data augmentation: compute vegetation indices and create image patches**

Data preprocessing, cleaning, and augmentation is often the most time-consuming part of a research workflow. This is a multi-step process that will depend on your research objectives and study design. We propose to compute vegetation indices from raw Sentinel bands and use them as input thematic layers. We selected:

- `Normalized Difference Vegetation Index (NDVI)` https://custom-scripts.sentinel-hub.com/custom-scripts/sentinel-2/ndvi/;
- `Red-edge Chlorophyll Index (CIre))` https://custom-scripts.sentinel-hub.com/custom-scripts/sentinel-2/chl_rededge/. 

In order to maximize the window of time where all fields have complete, cloud-free data, we have also trimmed our time series to the period from 2022-05-11 to 2022-10-09 (resulting in 31 field-level observations for each field sampled at a 5-day interval). Finally we have split our input field images into 3 by 3 patches. As a result, a single unit (patch) from this step is a 4D array with (31,2,3,3) shape. We save the output from this step for future use.

Below for demonstration we select 4 random fields (one of each crop) and plot the time series of their NDVI and CIre values aggregated to a field median to illustrate the differences between the growing seasons of different crops.

In [ ]:
# you can re-run this cell multiple times to plot a different random selection of fields
selected_fields = {
    crop: random.choice(
        list(training_labels_gdf[training_labels_gdf["CSB 2022"]==crop].index)) for crop in crop_list}
print ("Our random choices are : ", selected_fields)

In [ ]:
plot_indices_temporal(selected_fields = selected_fields,
                      filepath = raw_data_filename)

#### **Create training patches**

In [ ]:
%%time
create_patches(
    fids=train_ids,
    filepath=raw_data_filename,
    outfilepath=get_tmpdest(train_filename),
    patch_height=3, patch_width=3,
    allow_overlap=True,
    start_end_dates=["2022-05-11", "2022-10-09"]
)

#### **Create test patches**

In [ ]:
%%time
create_patches(
    fids=test_ids,
    filepath=raw_data_filename,
    outfilepath=get_tmpdest(test_filename),
    patch_height=3, patch_width=3,
    allow_overlap=True,
    start_end_dates=["2022-05-11", "2022-10-09"]
)

### **Step 3. Format/normalize input labels and observations**

#### **Format labels**
One hot encoding is needed to represent categorical variables in a machine learning model. After applying this technique to our labels, we load the train data and assign a one hot encoded label value to each patch. For more examples https://machinelearningmastery.com/why-one-hot-encode-data-in-machine-learning/. See an illustration below:

From the following input:

|category|
|--------|
| dog    |
| cat    |
| fox    |

We can generate the following one hot encoded representation:

| dog    | cat    | fox    |
|--------|--------|--------|
|**True**| False  | False  |
| False  |**True**| False  |
| False  | False  |**True**|

In [ ]:
one_hot_encoded_labels = pd.get_dummies(training_labels_gdf[["CSB 2022"]],
                                        columns = ["CSB 2022"], prefix_sep=" ") 
one_hot_encoded_labels.head()

#### **Assign data labels to data observations**

During this step, we are loading our train patches and labels to create our training dataset (`X_train`) and corresponding labels (`y_train`). This involves organizing the patches and their corresponding labels by pairing each patch with its corresponding label.

In [ ]:
train  = load_pickle(filepath=get_tmpdest(train_filename))
print(train.keys())

In [ ]:
X_train, y_train = generate_X_and_y(data=train, one_hot_encoded_labels=one_hot_encoded_labels)
print ("X shape: ", X_train.shape)
print ("y shape: ", y_train.shape)

#### **Normalize the input data using the `sklearn` MinMaxScaler**
We apply a scaler to each feature separately to normalize the data into the range `[0,1]`, so the minimum value of each feature becomes 0 and the maximum value becomes 1. This helps to ensure that no single feature dominates. Note: we fit the scaler to only `X_train` to be sure the model does not get information from the test set.

In [ ]:
X_train_scaled, scalers = apply_scaler_train(X_train=X_train)

In [ ]:
# check original and scaled min and max of your data to see if it worked
print (X_train.min(), X_train.max())
print (X_train_scaled.min(), X_train_scaled.max())

### **Step 4. Configure and train a model**

#### **Compile a model**
We compile Simple Recurrent Neural Network (RNN) model to process sequential (in our case time series) data input. For more information on model parameters follow `keras` documentation: https://keras.io/api/layers/recurrent_layers/simple_rnn/.

In [ ]:
model = compile_model(shape=(31, 2, 3, 3), kind="SimpleRNN")
print (model.summary())

#### **Compute class weights**
We can adjust the weight for the model to give more attention to the minority classes. This allows us to balance out under-represented (Sugarbeets) and over-represented (Soybeans) classes in the training set.

In [ ]:
# count the True values for each class type
y_train_counts = np.sum(y_train, axis=0)
print("Number of values for each class:", y_train_counts)

In [ ]:
class_weight = assign_class_weight(y=y_train)
print (class_weight)

#### **Fit the model**

During this process, we train the model using the train data and the parameters given. Our goal is to "teach" the model to generalize patterns in the data in order to predict a most likely outcome (crop type).

In [ ]:
%%time
history = model.fit(X_train_scaled, y_train,
                    batch_size=128, epochs=30, validation_split=0.2,
                    class_weight=class_weight)

### **Step 5. Make predictions and evaluate accuracy**
`X_test` and `y_test` need to go through the same preparatory steps as `X_train` and `y_train`.
- We load the test data from the `pickle` into a dictionary:

In [ ]:
test  = load_pickle(filepath=get_tmpdest(test_filename))
print(test.keys())

- We pair each test patch with the corresponding crop type label:

In [ ]:
X_test, y_test = generate_X_and_y(data=test, one_hot_encoded_labels=one_hot_encoded_labels)
print ("X shape: ", X_test.shape)
print ("y shape: ", y_test.shape)

- We apply scalers (previously generated using the train data) to adjust the range of values:

In [ ]:
X_test_scaled = apply_scaler_test(X_test=X_test, scalers=scalers)

In [ ]:
# check original and scaled min and max of your data to see if it worked
print (X_test.min(), X_test.max())
print (X_test_scaled.min(), X_test_scaled.max())

- We generate predictions. Note that the model does NOT output an exact label (crop type) but rather an array representing the model's estimated probabilities that the input belongs to each of the suggested classes. Therefore, `np.argmax` is applied to return the index corresponding to the highest probability.

In [ ]:
y_pred = model.predict(X_test_scaled)
y_pred = np.argmax(y_pred, axis=1)

- Here we format the true labels (`y_test`) for direct comparison with the model output:

In [ ]:
label_encoder = LabelEncoder()
y_true = label_encoder.fit_transform(np.argmax(y_test, axis=1))

- Finally, we use a **confusion matrix** as a performance measurement for our machine learning model. It is a great visual and quantitative way to show how many predictions are correct and incorrect for each class. It reveals which classes are more distinct and which ones are often confused by model as other classes, indicating lower separability.

In [ ]:
plot_confusion_matrix(y_true=y_true, y_pred=y_pred,
                      display_labels=["Corn", "Soybeans", "Spring Wheat", "Sugarbeets"])

- Below we also compute F1 score to evaluate overall model performance. It considers both precision and recall. The values of the F1 score range from 0 to 1 (higher values indicate better performace).

In [ ]:
F1 = f1_score(y_true, y_pred, average="weighted")
print ("F1 score is:", F1)

### **Step 6. Apply and test the model outside of the training area**

In this section we test if the model can be generalized to other counties in Minnesota.

- **Review new data.** We check the distribution of the new labels collected in a different county with a different crop profile.

In [ ]:
shapefile_data_filename_new = "chippewa-mn-csb-2022-training-set-wsg84.zip"

raw_data_filename_new = "chippewa_v2m58_plf_y2022_s{}.sr6d"

field_ids = list(range(0,280))

test_filename_new = "test_patches_new.sr6d"

In [ ]:
training_labels_gdf_new = gpd.read_file(
    download_file(bucket_path="csb/", filename=shapefile_data_filename_new))
print ("Count of agricultural fields:", len(training_labels_gdf_new))
training_labels_gdf_new.head()

In [ ]:
map_training_labels(gdf=training_labels_gdf_new, column_name="CSB 2022")

In [ ]:
plot_sample_distribution(gdf=training_labels_gdf_new, column_name="CSB 2022")

- **Create new test patches**

In [ ]:
%%time
create_patches(
    fids=field_ids,
    filepath=raw_data_filename_new,
    outfilepath=get_tmpdest(test_filename_new),
    patch_height=3, patch_width=3,
    allow_overlap=False,
    start_end_dates=["2022-05-11", "2022-10-11"]
)

- **Format labels**

In [ ]:
one_hot_encoded_labels_new = pd.get_dummies(training_labels_gdf_new[["CSB 2022"]],
                                            columns = ["CSB 2022"], prefix_sep=" ") 
one_hot_encoded_labels_new.head()

- **Assign data labels to data observations**

In [ ]:
test_new  = load_pickle(filepath=get_tmpdest(test_filename_new))
print(test_new.keys())

In [ ]:
X_test_new, y_test_new = generate_X_and_y(data=test_new, one_hot_encoded_labels=one_hot_encoded_labels_new)
print ("X shape: ", X_test_new.shape)
print ("y shape: ", y_test_new.shape)

- **Normalize the input data using scalers**

In [ ]:
X_test_scaled_new = apply_scaler_test(X_test=X_test_new, scalers=scalers)

In [ ]:
# check original and scaled min and max of your data to see if it worked
print (X_test_new.min(), X_test_new.max())
print (X_test_scaled_new.min(), X_test_scaled_new.max())

- **Generate predictions**

In [ ]:
y_pred_new = model.predict(X_test_scaled_new)
y_pred_new = np.argmax(y_pred_new, axis=1)

- **Accuracy assessment**

In [ ]:
label_encoder = LabelEncoder()
y_true_new = label_encoder.fit_transform(np.argmax(y_test_new, axis=1))

In [ ]:
plot_confusion_matrix(y_true=y_true_new, y_pred=y_pred_new,
                      display_labels=["Corn", "Soybeans", "Spring Wheat", "Sugarbeets"])

In [ ]:
F1 = f1_score(y_true_new, y_pred_new, average="weighted")
print ("F1 score is:", F1)